<a href="https://colab.research.google.com/github/Mihirkool/JP-Morgan-Qualitative-analysis/blob/main/Task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
uploaded = files.upload()


Saving Task 3 and 4_Loan_Data.csv to Task 3 and 4_Loan_Data.csv


In [7]:
import pandas as pd

# Adjust if the file name changes after upload
df = pd.read_csv("Task 3 and 4_Loan_Data.csv")
df.head()


,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


i will perform the first step now let ssee ;)

In [8]:
import pandas as pd
import numpy as np

# Number of buckets
num_buckets = 5

# Create quantile-based FICO buckets
df['fico_bucket'], bucket_bins = pd.qcut(df['fico_score'], q=num_buckets, retbins=True, labels=False, duplicates='drop')

# Compute PD (probability of default) per bucket
bucket_stats = df.groupby('fico_bucket').agg(
    avg_fico=('fico_score', 'mean'),
    count=('default', 'count'),
    defaults=('default', 'sum')
)

bucket_stats['PD'] = bucket_stats['defaults'] / bucket_stats['count']
bucket_stats = bucket_stats.reset_index()

bucket_stats


,fico_bucket,avg_fico,count,defaults,PD
0,0,552.611707,2050,817,0.398537
1,1,606.751395,1971,425,0.215627
2,2,638.579688,1989,301,0.151332
3,3,670.346520,1997,200,0.100150
4,4,721.524837,1993,108,0.054190


Now the second step ;)
Expected Loss=PD×(1−Recovery Rate)×Loan Amount

In [9]:
# Set recovery rate
recovery_rate = 0.10
LGD = 1 - recovery_rate

# Build a list of bucket boundaries and PDs
bucket_edges = bucket_bins
bucket_pds = bucket_stats['PD'].values

# Function to get expected loss
def estimate_expected_loss(fico_score, loan_amount):
    # Find which bucket the FICO score falls into
    for i in range(len(bucket_edges) - 1):
        if bucket_edges[i] <= fico_score < bucket_edges[i + 1]:
            pd = bucket_pds[i]
            break
    else:
        # Handle edge case: score exactly equals max score
        pd = bucket_pds[-1]

    # Calculate Expected Loss
    expected_loss = pd * LGD * loan_amount
    return expected_loss


Testing the function


In [10]:
# Example
fico_input = 680
loan_input = 100000

estimate_expected_loss(fico_input, loan_input)


np.float64(9013.520280420631)